In [ ]:
import datetime
from pprint import pprint

import japanize_matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import yfinance as yf
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from stockstats import StockDataFrame

sns.set(font="IPAexGothic", rc={"figure.figsize": (11, 8)})
pd.options.display.float_format = "{:6.2f}".format

/usr/local/lib/python3.9/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [ ]:
# Valid start and end: YYYY-MM-DD
# Valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
# Valid intervals: [1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, 3mo]
response = yf.download(
    tickers="NDAQ",
    period="5y",
    interval="1d",
    group_by="ticker",
)

[*********************100%***********************]  1 of 1 completed


In [ ]:
yfdata = response.copy().dropna()
# yfdata = yfdata["1950-01":"202１-12"]  # 直近の暴落を除いて検証する
yfdata

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2017-09-05,25.12,25.12,24.57,24.79,22.80,3149700
2017-09-06,24.88,25.03,24.72,24.73,22.74,2398500
2017-09-07,24.74,24.78,24.45,24.57,22.59,3294300
2017-09-08,24.58,24.92,24.58,24.78,22.79,2528400
2017-09-11,24.88,25.05,24.69,24.72,22.73,4331100
...,...,...,...,...,...,...
2022-08-29,60.32,60.82,59.64,59.70,59.70,1743000
2022-08-30,60.16,60.45,59.37,59.60,59.60,2037700
2022-08-31,59.90,60.26,59.34,59.53,59.53,3367400


In [ ]:
def convert_df_to_stock_df(df: pd.DataFrame) -> StockDataFrame:
    sdf = df.copy()
    sdf.rename(
        columns={
            "Open": "open",
            "High": "high",
            "Low": "low",
            "Close": "close",
            "Adj Close": "amount",
            "Volume": "volume",
        },
        inplace=True,
    )
    sdf.index.names = ["date"]
    return StockDataFrame(sdf)


def MACD(
    arr: pd.DataFrame, short: int = 12, long: int = 26
) -> tuple[pd.Series, pd.Series]:
    sdf = convert_df_to_stock_df(arr)
    StockDataFrame.MACD_EMA_SHORT = short
    StockDataFrame.MACD_EMA_LONG = long
    StockDataFrame.MACD_EMA_SIGNAL = 9
    return (sdf["macd"], sdf["macds"])


def RSI(arr: pd.DataFrame, rsi: int) -> pd.Series:
    sdf = convert_df_to_stock_df(arr)
    return sdf["rsi_" + str(rsi)]


def RCI(arr: pd.DataFrame, rsi: int) -> pd.Series:
    sdf = convert_df_to_stock_df(arr)
    return sdf["rsi_" + str(rsi)]


# 週足に変換する
def day_2_week(pd: pd.DataFrame) -> pd.DataFrame:
    return (
        pd.copy()
        .resample("W")
        .agg(
            {
                "Open": "first",
                "High": "max",
                "Low": "min",
                "Close": "last",
                "Volume": "sum",
            }
        )
    )


def add_html(file: str, dom: str, first_insert: bool) -> None:
    with open(file) as reader:
        r = reader.read()

    if first_insert:
        r = dom + r
    else:
        r = r + dom

    with open(file, "w") as writer:
        writer.write(r)

In [ ]:
def macd_week(data: pd.DataFrame) -> tuple[pd.Series, pd.Series]:
    m, s = MACD(day_2_week(yfdata))

    # 週間隔のデータなので、日ごとにする
    df = pd.concat([data.copy(), pd.DataFrame([m, s]).transpose()], axis=1)

    tmp_macd = 0
    tmp_macds = 0

    # 一週間に１日だけしかデータが入っていないので、直近のデータをコピーする
    for index, row in df.iterrows():
        if pd.isna(row["macd"]) or pd.isna(row["macds"]):
            df.at[index, "macd"] = tmp_macd
            df.at[index, "macds"] = tmp_macds
        else:
            tmp_macd = row["macd"]
            tmp_macds = row["macds"]

    df = df.dropna()
    return (df["macd"], df["macds"])


def rsi_week(data: pd.DataFrame, during: int) -> pd.Series:
    rsi = RSI(day_2_week(yfdata), during)
    df = pd.concat([data.copy(), pd.DataFrame([rsi]).transpose()], axis=1)

    tmp_rsi = 0
    key = "rsi_" + str(during)
    for index, row in df.iterrows():
        if pd.isna(row[key]):
            df.at[index, key] = tmp_rsi
        else:
            tmp_rsi = row[key]

    df = df.dropna()
    return df[key]

In [ ]:
class My_Strategy(Strategy):
    # RSI
    sort_macd_ema_short = 12
    sort_macd_ema_long = 26

    long_macd_ema_short = 20
    long_macd_ema_long = 7

    wait_long_crros_time = 7
    wait_long_crros_count = 0
    is_long_macd_crrosed = False

    def init(self):
        self.short_macd, self.short_macd_signal = self.I(
            MACD, self.data.df, self.sort_macd_ema_short, self.sort_macd_ema_long
        )
        self.long_macd, self.long_macd_signal = self.I(
            MACD, self.data.df, self.long_macd_ema_short, self.long_macd_ema_long
        )
        # self.rsi = self.I(RSI, self.data.df, self.prop_rsi)

    def golden_cross_with_short_macd(self):
        return crossover(self.short_macd, self.short_macd_signal)

    def golden_cross_with_long_macd(self):
        return crossover(self.long_macd, self.long_macd_signal)

    def dead_cross_with_short_macd(self):
        return crossover(self.short_macd_signal, self.short_macd)

    def dead_cross_with_long_macd(self):
        return crossover(self.long_macd_signal, self.long_macd)

    def reset_wait_info(self):
        self.wait_long_crros_count = 0
        self.is_long_macd_crrosed = False

    def next(self):
        # 計算できていない場合トレードしない
        if len(self.data.index) < 52:
            return

        # MACDがゴールデンクロスしたら、今までの注文を終了して買い注文
        if self.golden_cross_with_short_macd():
            self.reset_wait_info()
            self.position.close()
            self.buy()
            return

        # MACDがデッドクロスしたら、今までの注文を終了して売り注文
        if self.dead_cross_with_short_macd():
            self.reset_wait_info()
            self.position.close()
            self.sell()
            return

        if (
            self.position.is_long or self.position.is_long
        ) and self.is_long_macd_crrosed == False:
            if self.dead_cross_with_long_macd() or self.golden_cross_with_long_macd():
                self.is_long_macd_crrosed = True

            # 指定日数経ってもロングのMacdがクロスしなかったら損切り
            if self.wait_long_crros_count == self.wait_long_crros_time:
                self.reset_wait_info()
                self.position.close()

            self.wait_long_crros_count += 1


bt = Backtest(yfdata, My_Strategy, cash=10000, commission=0.002, exclusive_orders=True)


# 最適化
# #dayly : 127%
optimize = bt.optimize(
    # sort_macd_ema_short=range(5, 55, 10),
    # sort_macd_ema_long=range(5, 55, 10),
    long_macd_ema_short=range(20, 80, 10),
    long_macd_ema_long=range(20, 80, 10),
    # wait_long_crros_time=range(1, 10, 1),
    constraint=lambda p:
    # p.sort_macd_ema_long < p.sort_macd_ema_long or
    p.long_macd_ema_short < p.long_macd_ema_long,
    # or p.sort_macd_ema_long <= p.long_macd_ema_short,
    # method="grid",  # unuse model-based optimization
    maximize="SQN",
)
bt.plot()
print(optimize)
print(optimize._strategy)


# 出力
# output = bt.run()
# print("Return : " + str(output["Return [%]"]) + "%")
# print("Trades : " + str(output["# Trades"]))
# print(output)
# bt.plot()

Start                     2017-09-05 00:00:00
End                       2022-09-02 00:00:00
Duration                   1823 days 00:00:00
Exposure Time [%]                       88.96
Equity Final [$]                     15318.61
Equity Peak [$]                      16526.28
Return [%]                              53.19
Buy & Hold Return [%]                  140.71
Return (Ann.) [%]                        8.91
Volatility (Ann.) [%]                   24.74
Sharpe Ratio                             0.36
Sortino Ratio                            0.58
Calmar Ratio                             0.28
Max. Drawdown [%]                      -32.37
Avg. Drawdown [%]                       -6.23
Max. Drawdown Duration      858 days 00:00:00
Avg. Drawdown Duration       74 days 00:00:00
# Trades                                   85
Win Rate [%]                            43.53
Best Trade [%]                          18.00
Worst Trade [%]                         -9.66
Avg. Trade [%]                    

In [ ]:
# write HTML file
filename = "Return_" + str(round(output["Return [%]"])) + "%"
bt.plot(filename=filename)

Row(id='24792', ...)